# Chapter 1 신경망 복습
## 1.1 수학과 파이썬 복습
### 1.1.1 벡터와 행렬

In [1]:
import numpy as np
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

In [2]:

x = np.array([1,2,3])
x.__class__
x.shape
x.ndim

W = np.array([[1,2,3],[4,5,6]])
W.shape
W.ndim

2

### 1.1.2 행렬의 원소별 연산

In [3]:
W = np.array([[1,2,3],[4,5,6]])
X = np.array([[0,1,2],[3,4,5]])
W+X
W*X

array([[ 0,  2,  6],
       [12, 20, 30]])

### 1.1.3 브로드캐스트

In [4]:
A = np.array([[1,2],[3,4]])
A * 10

array([[10, 20],
       [30, 40]])

In [5]:
A = np.array([[1,2],[3,4]])
b = np.array([[10,20]])
A*b

array([[10, 40],
       [30, 80]])

### 1.1.4 벡터의 내적과 행렬의 곱

In [6]:
# 벡터의 내적
a = np.array([1,2,3])
b = np.array([4,5,6])
np.dot(a,b)

#행렬의 곱
A = np.array([[1,2],[3,4]])
B = np.array([[5,6],[7,8]])
np.matmul(A, B)


array([[19, 22],
       [43, 50]])

## 1.2 신경망의 추론
### 1.2.1 신경망 추론 전체 그림

In [7]:
import numpy as np


def sigmoid(x):
    return 1 / (1 + np.exp(-x))

x = np.random.randn(10, 2) #입력
x
W1 = np.random.randn(2, 4) # 가중치
b1 = np.random.randn(4) # 편향
W2 = np.random.randn(4,3)
b2 = np.random.randn(3)


h = np.matmul(x, W1) + b1
a = sigmoid(h)
s = np.matmul(a, W2) + b2

h
a
s

array([[-1.06842853, -1.45368135,  0.29474041],
       [-0.61217487, -2.39279609, -0.98284309],
       [-1.02970088, -1.42974639,  0.12329004],
       [-1.10439496, -1.41715459,  0.16921706],
       [-0.90356279, -1.50880027, -0.03259065],
       [-0.58927939, -2.44281702, -1.02458698],
       [-1.13887926, -1.45542589,  0.53262092],
       [-0.87236636, -1.54405335, -0.06973933],
       [-0.77478699, -1.76047891, -0.19757826],
       [-0.92342682, -2.11275123, -0.07011144]])

### 1.2.2 계층으로 클래스화 및 순전파 구현

In [8]:
import numpy as np

class Sigmoid:
    def __init__(self):
        self.params = []

    def forward(self, x):
        return 1 / (1 + np.exp(-x))
    
class Affine:
    def __init__(self, W, b):
        self.params = [W, b]
    
    def forward(self, x):
        W, b = self.params
        print("W ",W.shape)
        print("b: ",b.shape)
        print("x: ",x.shape)
        out = np.matmul(x, W) + b
        return out
    

In [9]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size):
        I, H, O = input_size, hidden_size, output_size

        # 가중치와 편향 초기화
        W1 = np.random.randn(I, H)
        b1 = np.random.randn(H)
        W2 = np.random.randn(H, O)
        b2 = np.random.randn(O)

        # 계층 생성
        self.layers = [
            Affine(W1, b1),
            Sigmoid(),
            Affine(W2, b2)
        ]

        # 모든 가중치를 리스트에 모은다. 
        self.params = []
        for layer in self.layers:
            self.params += layer.params
    
    def predict(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x


x = np.random.randn(10, 2)
model = TwoLayerNet(2, 4, 3)
s = model.predict(x)

W  (2, 4)
b:  (4,)
x:  (10, 2)
W  (4, 3)
b:  (3,)
x:  (10, 4)


## 1.3 신경망의 학습
### 1.3.4 계산 그래프

In [10]:
class MatMul:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None
    
    def forward(self, x):
        W, = self.params
        out = np.matmul(x, W)
        self.x = x
        return out
    
    def backward(self, dout):
        W, self.params
        dx = np.matmul(dout, W.T)
        dW = np.matmul(self.x.T, dout)
        self.grads[0][...] = dW
        return dx
    

### 1.3.5 기울기 도출과 역전파 구현

In [11]:
class Sigmoid:
    def __init__(self):
        self.params, self.grads = [], []
        self.out = None
    
    def forward(self, x):
        out = 1/ (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx 

In [12]:
class Affine:
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None

    def forward(self, x):
        W, b = self.params
        out = np.matmul(x, W) + b
        self.x = x
    
    def backward(self, dout):
        W, b = self.params
        dx = np.matmul(dout, W.T)
        dW = np.matmul(self.x.T, dout)
        db = np.sum(dout, axis=0)

        self.grads[0][...] = dW
        self.grads[1][...] = db
        return dx
    

### 1.3.6 가중치 갱신

In [13]:
class SGD:
    def __init__(self, lr = 0.01):
        self.lr = lr
    
    def update(self, params, grads):
        for i in range(len(params)):
            params[i] -= self.lr * grads[i]

            

## 1.4 신경망으로 문제를 풀다
### 1.4.1 스파이럴 데이터셋

In [14]:
import sys
sys.path.append('..')
from dataset import spiral
import matplotlib.pyplot as plt

x, t = spiral.load_data()
print('x', x.shape)
print('t', t.shape)

x (300, 2)
t (300, 3)


### 1.4.2 신경망 구현

In [15]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import Affine, Sigmoid, SoftmaxWithLoss

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size):
        I, H, O = input_size, hidden_size, output_size

        # 가중치와 편향 초기화
        W1 = 0.01 * np.random.randn(I, H)
        b1 = np.zeros(H)
        W2 = 0.01 * np.random.randn(H, O)
        b2 = np.zeros(O)

        # 계층 생성
        self.layers = [
            Affine(W1, b1),
            Sigmoid(),
            Affine(W2, b2)
        ]
        self.loss_layer = SoftmaxWithLoss()

        # 모든 가중치와 기울기기를 리스트에 모은다. 
        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads

        
    def predict(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def forward(self, x, t):
        score = self.predict(x)
        loss = self.loss_layer.forward(score, t)
        return loss
    
    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)

        return dout

### 1.4.3 학습용 코드

In [16]:

import sys
sys.path.append('..')
import numpy as np
from common.optimizer import SGD
from dataset import spiral
import matplotlib.pyplot as plt
# from two_layer_net import TwoLayerNet

# 하이퍼파라미터 설정
max_epoch = 300
batch_size = 30
hidden_size = 10
learning_rate = 1.0

# 데이터 읽기, 모델과 옵티마이저 생성
x, t = spiral.load_data()
model = TwoLayerNet(input_size=2, hidden_size=hidden_size, output_size=3)
optimizer = SGD(lr=learning_rate)

# 학습에 사용하는 변수
data_size = len(x)
max_iters = data_size // batch_size
total_loss = 0 
loss_count = 0
loss_list = []

for epoch in range(max_epoch):
    # 데이터 뒤섞기
    idx = np.random.permutation(data_size)
    x = x[idx]
    t = t[idx]

    for iters in range(max_iters):
        batch_x = x[iters*batch_size:(iters+1)*batch_size]
        batch_t = t[iters*batch_size:(iters+1)*batch_size]
        
        # 기울기를 구해 매개변수 갱신
        loss = model.forward(batch_x, batch_t)
        model.backward()
        optimizer.update(model.params, model.grads)

        total_loss += loss
        loss_count += 1

        # 정기적으로 학습 경과 출력
        if (iters+1) % 10 == 0:
            avg_loss = total_loss / loss_count
            print('| 에폭 %d | 반복 %d | %d |손실 %.2f'
                  % (epoch +1 , iters +1, max_iters, avg_loss))
            loss_list.append(avg_loss)
            total_loss, loss_count = 0, 0



| 에폭 1 | 반복 10 | 10 |손실 1.13
| 에폭 2 | 반복 10 | 10 |손실 1.13
| 에폭 3 | 반복 10 | 10 |손실 1.12
| 에폭 4 | 반복 10 | 10 |손실 1.12
| 에폭 5 | 반복 10 | 10 |손실 1.11
| 에폭 6 | 반복 10 | 10 |손실 1.14
| 에폭 7 | 반복 10 | 10 |손실 1.16
| 에폭 8 | 반복 10 | 10 |손실 1.11
| 에폭 9 | 반복 10 | 10 |손실 1.12
| 에폭 10 | 반복 10 | 10 |손실 1.13
| 에폭 11 | 반복 10 | 10 |손실 1.12
| 에폭 12 | 반복 10 | 10 |손실 1.11
| 에폭 13 | 반복 10 | 10 |손실 1.09
| 에폭 14 | 반복 10 | 10 |손실 1.08
| 에폭 15 | 반복 10 | 10 |손실 1.04
| 에폭 16 | 반복 10 | 10 |손실 1.03
| 에폭 17 | 반복 10 | 10 |손실 0.96
| 에폭 18 | 반복 10 | 10 |손실 0.92
| 에폭 19 | 반복 10 | 10 |손실 0.92
| 에폭 20 | 반복 10 | 10 |손실 0.87
| 에폭 21 | 반복 10 | 10 |손실 0.85
| 에폭 22 | 반복 10 | 10 |손실 0.82
| 에폭 23 | 반복 10 | 10 |손실 0.79
| 에폭 24 | 반복 10 | 10 |손실 0.78
| 에폭 25 | 반복 10 | 10 |손실 0.82
| 에폭 26 | 반복 10 | 10 |손실 0.78
| 에폭 27 | 반복 10 | 10 |손실 0.76
| 에폭 28 | 반복 10 | 10 |손실 0.76
| 에폭 29 | 반복 10 | 10 |손실 0.78
| 에폭 30 | 반복 10 | 10 |손실 0.75
| 에폭 31 | 반복 10 | 10 |손실 0.78
| 에폭 32 | 반복 10 | 10 |손실 0.77
| 에폭 33 | 반복 10 | 10 |손실 0.77
| 에폭 34 | 반복 10 | 1